In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
from tqdm import tqdm
importlib.reload(ass)
import ast

ERROR:root:Cannot activate multiple GUI eventloops


In [17]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [120]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\proAnubis_240403_0227.h5')

In [121]:
thisData = [sublist[:94230] for sublist in thisData]

In [122]:
etaHits, phiHits = ass.divideHitCountsByRPC_Timed(thisData)

In [123]:
df = ass.remake_data(thisData, mapping)

In [124]:
df_clean = df[df['time'] <= 500]

In [125]:
cluster_df = ass.calculate_cluster_metrics_better(df_clean, 5)

In [126]:
def filter_groups(group):
    has_eta = 'eta' in group['strip_direction'].values
    has_phi = 'phi' in group['strip_direction'].values
    return has_eta and has_phi

filtered_cluster = cluster_df.groupby(['event_number', 'rpc_number']).filter(filter_groups)

In [13]:
print(filtered_cluster.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697379 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     697379 non-null  int64 
 1   rpc_number       697379 non-null  object
 2   strip_direction  697379 non-null  object
 3   locations        697379 non-null  object
 4   times            697379 non-null  object
 5   start_time       697379 non-null  int64 
 6   end_time         697379 non-null  int64 
 7   size             697379 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 47.9+ MB
None


In [18]:
show(filtered_cluster.sort_values(by=['event_number', 'rpc_number']))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [10]:
print(cluster_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839727 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     839727 non-null  int64 
 1   rpc_number       839727 non-null  object
 2   strip_direction  839727 non-null  object
 3   locations        839727 non-null  object
 4   times            839727 non-null  object
 5   start_time       839727 non-null  int64 
 6   end_time         839727 non-null  int64 
 7   size             839727 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 51.3+ MB
None


In [127]:
fully_massaged_df = filtered_cluster[filtered_cluster['size'] <= 5]

In [16]:
print(fully_massaged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675851 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     675851 non-null  int64 
 1   rpc_number       675851 non-null  object
 2   strip_direction  675851 non-null  object
 3   locations        675851 non-null  object
 4   times            675851 non-null  object
 5   start_time       675851 non-null  int64 
 6   end_time         675851 non-null  int64 
 7   size             675851 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 46.4+ MB
None


In [33]:
show(fully_massaged_df.sort_values(by = ['event_number']))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [ ]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / uncertainties**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        # cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            
            for combo in combinations(all_clusters, n):
                if len({c['rpc'] for c in combo}) != len(combo):
                    continue  # Skip combinations where RPCs are not unique

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                        uncertainty_margin = 5

                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if not time_diffs_are_valid:
                    continue  # Skip to the next combination if time differences are invalid

                # Calculate metrics only for valid combinations
                metric = calculate_metric_for_combo(combo, rpc_heights)
                if metric[4] <= 10:
                    valid_combinations.append(metric)
                    

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [128]:
df_no_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc0']
df_no_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc1']
df_no_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc2']
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']
df_no_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc4']
df_no_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc5']


In [ ]:
df_recon_no_rpc0_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc0, 5)
df_recon_no_rpc1_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc1, 5)
df_recon_no_rpc2_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc2, 5)
df_recon_no_rpc3_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)
df_recon_no_rpc4_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc4, 5)
df_recon_no_rpc5_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc5, 5)

Processing Events: 100%|██████████| 92919/92919 [01:17<00:00, 1196.73it/s]


In [ ]:
df_recon_no_rpc0_4.to_excel('df_recon_no_rpc0_5_AtlasOFF.xlsx')
df_recon_no_rpc1_4.to_excel('df_recon_no_rpc1_5_AtlasOFF.xlsx')
df_recon_no_rpc2_4.to_excel('df_recon_no_rpc2_5_AtlasOFF.xlsx')
df_recon_no_rpc3_4.to_excel('df_recon_no_rpc3_5_AtlasOFF.xlsx')
df_recon_no_rpc4_4.to_excel('df_recon_no_rpc4_5_AtlasOFF.xlsx')
df_recon_no_rpc5_4.to_excel('df_recon_no_rpc5_5_AtlasOFF.xlsx')

In [129]:
df_recon_no_rpc0_4 = pd.read_excel('df_recon_no_rpc0_5_AtlasON.xlsx')
df_recon_no_rpc1_4 = pd.read_excel('df_recon_no_rpc1_5_AtlasON.xlsx')
df_recon_no_rpc2_4 = pd.read_excel('df_recon_no_rpc2_5_AtlasON.xlsx')
df_recon_no_rpc3_4 = pd.read_excel('df_recon_no_rpc3_5_AtlasON.xlsx')
df_recon_no_rpc4_4 = pd.read_excel('df_recon_no_rpc4_5_AtlasON.xlsx')
df_recon_no_rpc5_4 = pd.read_excel('df_recon_no_rpc5_5_AtlasON.xlsx')

In [129]:
print(df_recon_no_rpc0_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      1253 non-null   int64  
 1   Direction         1253 non-null   object 
 2   Slope             1253 non-null   float64
 3   Intercept         1253 non-null   float64
 4   Slope_error       1253 non-null   int64  
 5   Intercept_error   1253 non-null   int64  
 6   Used Coordinates  1253 non-null   object 
 7   RSS               1253 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 78.4+ KB
None


In [75]:
show(df_recon_no_rpc0_4.sort_values('Event Number'))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [130]:
df_recon_no_rpc0_4_filtered = df_recon_no_rpc0_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
              .reset_index(drop=True)
              
df_recon_no_rpc1_4_filtered = df_recon_no_rpc1_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
                                .reset_index(drop=True)
df_recon_no_rpc2_4_filtered = df_recon_no_rpc2_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
              
df_recon_no_rpc3_4_filtered = df_recon_no_rpc3_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc4_4_filtered = df_recon_no_rpc4_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc5_4_filtered = df_recon_no_rpc5_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)

In [74]:
show(df_recon_no_rpc0_4_filtered.reset_index(drop=True))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [131]:
df_recon_no_rpc0_4_filtered['expected hits'] = df_recon_no_rpc0_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc1_4_filtered['expected hits'] = df_recon_no_rpc1_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc2_4_filtered['expected hits'] = df_recon_no_rpc2_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc3_4_filtered['expected hits'] = df_recon_no_rpc3_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc4_4_filtered['expected hits'] = df_recon_no_rpc4_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc5_4_filtered['expected hits'] = df_recon_no_rpc5_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)

In [132]:
fully_massaged_df_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc0']
fully_massaged_df_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc1']
fully_massaged_df_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc2']
fully_massaged_df_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc3']
fully_massaged_df_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc4']
fully_massaged_df_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc5']

def calculate_midpoint(locations, direction):
    if direction == 'eta':
        return np.mean([loc[1] for loc in locations])
    elif direction == 'phi':
        return np.mean([loc[0] for loc in locations])
    return None

fully_massaged_df_rpc0['midpoint'] = fully_massaged_df_rpc0.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc1['midpoint'] = fully_massaged_df_rpc1.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc2['midpoint'] = fully_massaged_df_rpc2.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc3['midpoint'] = fully_massaged_df_rpc3.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc4['midpoint'] = fully_massaged_df_rpc4.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc5['midpoint'] = fully_massaged_df_rpc5.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)

merged_df0 = pd.merge(df_recon_no_rpc0_4_filtered, fully_massaged_df_rpc0, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df1 = pd.merge(df_recon_no_rpc1_4_filtered, fully_massaged_df_rpc1, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df2 = pd.merge(df_recon_no_rpc2_4_filtered, fully_massaged_df_rpc2, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df3 = pd.merge(df_recon_no_rpc3_4_filtered, fully_massaged_df_rpc3, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df4 = pd.merge(df_recon_no_rpc4_4_filtered, fully_massaged_df_rpc4, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df5 = pd.merge(df_recon_no_rpc5_4_filtered, fully_massaged_df_rpc5, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')


merged_df0['hit_difference'] = merged_df0['expected hits'] - merged_df0['midpoint']
merged_df1['hit_difference'] = merged_df1['expected hits'] - merged_df1['midpoint']
merged_df2['hit_difference'] = merged_df2['expected hits'] - merged_df2['midpoint']
merged_df3['hit_difference'] = merged_df3['expected hits'] - merged_df3['midpoint']
merged_df4['hit_difference'] = merged_df4['expected hits'] - merged_df4['midpoint']
merged_df5['hit_difference'] = merged_df5['expected hits'] - merged_df5['midpoint']

result_df0 = merged_df0[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df1 = merged_df1[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df2 = merged_df2[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df3 = merged_df3[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df4 = merged_df4[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df5 = merged_df5[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]


C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [133]:
eta_df0 = result_df0[result_df0['Direction'] == 'eta']
eta_df1 = result_df1[result_df1['Direction'] == 'eta']
eta_df2 = result_df2[result_df2['Direction'] == 'eta']
eta_df3 = result_df3[result_df3['Direction'] == 'eta']
eta_df4 = result_df4[result_df4['Direction'] == 'eta']
eta_df5 = result_df5[result_df5['Direction'] == 'eta']

phi_df0 = result_df0[result_df0['Direction'] == 'phi']
phi_df1 = result_df1[result_df1['Direction'] == 'phi']
phi_df2 = result_df2[result_df2['Direction'] == 'phi']
phi_df3 = result_df3[result_df3['Direction'] == 'phi']
phi_df4 = result_df4[result_df4['Direction'] == 'phi']
phi_df5 = result_df5[result_df5['Direction'] == 'phi']

In [134]:
def get_min_index(group):
    if group['hit_difference'].isna().all():
        return group.index.tolist() 
    return [group['hit_difference'].idxmin()]


In [135]:
phi_df0 = phi_df0.loc[phi_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df1 = phi_df1.loc[phi_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df2 = phi_df2.loc[phi_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df3 = phi_df3.loc[phi_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df4 = phi_df4.loc[phi_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df5 = phi_df5.loc[phi_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]

eta_df0 = eta_df0.loc[eta_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df1 = eta_df1.loc[eta_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df2 = eta_df2.loc[eta_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df3 = eta_df3.loc[eta_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df4 = eta_df4.loc[eta_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df5 = eta_df5.loc[eta_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]



In [138]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [118]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [119]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(6):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(6):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]
        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

In [31]:
show(combined_df[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [32]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Chi2 Analysis

In [149]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1.333

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], histtype='step', linewidth=1.3, log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas Off')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [177]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas ON')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [155]:
print(df_recon_no_rpc0_4_filtered.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1132 non-null   int64  
 1   Event Number      1132 non-null   int64  
 2   Direction         1132 non-null   object 
 3   Slope             1132 non-null   float64
 4   Intercept         1132 non-null   float64
 5   Slope_error       1132 non-null   int64  
 6   Intercept_error   1132 non-null   int64  
 7   Used Coordinates  1132 non-null   object 
 8   RSS               1132 non-null   float64
 9   Chi_squared       1132 non-null   float64
 10  expected hits     1132 non-null   float64
dtypes: float64(5), int64(4), object(2)
memory usage: 97.4+ KB
None


In [12]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']
df_eta_list = []
df_phi_list = []
                    
Variance_eta = ( 0.5 * 3.09375) ** 2 / 12
Variance_phi = ( 0.5 * 2.8125) ** 2 / 12

for df in df_chi2:
    df_eta = df[df['Direction'] == 'eta']
    df_phi = df[df['Direction'] == 'phi']
    
    # Append to lists
    df_eta_list.append(df_eta)
    df_phi_list.append(df_phi)

# Calculate Chi_squared values
for df in df_eta_list:
    df['Chi_squared'] = df['RSS'] / Variance_eta 
for df in df_phi_list:
    df['Chi_squared'] = df['RSS'] / Variance_phi 

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_eta_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log = True)


plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Eta Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_phi_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log=True)

plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Phi Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_eta  # This is the same as just using RSS directly
C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_phi  # This is the same as just using RSS directly


In [187]:
show(df_eta_list[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [151]:
show(df_recon_no_rpc0_4_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [13]:
show(eta_df0)

NameError: name 'eta_df0' is not defined

In [34]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]


efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [37]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [38]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(3):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(3):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]

        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

Separate them into TDCs Using a reverse Mapping

In [43]:
show(phi_df1)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [136]:
reversed_mapping = {
    ('rpc0', 'eta'): [(0, 32, 0)],
    ('rpc0', 'phi'): [(0, 64, 0)],
    ('rpc1', 'eta'): [(0, 32, 0)],
    ('rpc1', 'phi'): [(0, 64, 1)],
    ('rpc2', 'eta'): [(0, 32, 1)],
    ('rpc2', 'phi'): [(0, 64, 1)],
    ('rpc3', 'eta'): [(0, 32, 2)],
    ('rpc3', 'phi'): [(0, 64, 2)],
    ('rpc4', 'eta'): [(0, 32, 3)],
    ('rpc4', 'phi'): [(0, 64, 3)],
    ('rpc5', 'eta'): [(0, 32, 3)],
    ('rpc5', 'phi'): [(0, 64, 4)]
}

for key, value in reversed_mapping.items():
    print(key, ":", value)
    
def compute_tdc_number(rpc, direction, expected_hits, mapping):
    if (rpc, direction) in mapping:
        for (start, end, chip) in mapping[(rpc, direction)]:  

            if start <= expected_hits <= end:

                return int(chip)
    return None  

print(compute_tdc_number('rpc1', 'eta', 12.021, reversed_mapping))

('rpc0', 'eta') : [(0, 32, 0)]
('rpc0', 'phi') : [(0, 64, 0)]
('rpc1', 'eta') : [(0, 32, 0)]
('rpc1', 'phi') : [(0, 64, 1)]
('rpc2', 'eta') : [(0, 32, 1)]
('rpc2', 'phi') : [(0, 64, 1)]
('rpc3', 'eta') : [(0, 32, 2)]
('rpc3', 'phi') : [(0, 64, 2)]
('rpc4', 'eta') : [(0, 32, 3)]
('rpc4', 'phi') : [(0, 64, 3)]
('rpc5', 'eta') : [(0, 32, 3)]
('rpc5', 'phi') : [(0, 64, 4)]
0


In [64]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [137]:
eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]
phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]
rpc_number = ['rpc0', 'rpc1', 'rpc2', 'rpc3', 'rpc4', 'rpc5']
for i in range(6):
    eta_df[i]['TDC Number'] = eta_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], -row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )
    phi_df[i]['TDC Number'] = phi_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )

    

In [138]:
combined_eta_df = pd.concat(eta_df, ignore_index=True)
combined_phi_df = pd.concat(phi_df, ignore_index=True)

# Now concatenate the two resulting DataFrames vertically as well
combined_df = pd.concat([combined_eta_df, combined_phi_df], ignore_index=True)

In [111]:
show(combined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [112]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7542 entries, 0 to 7541
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Number    7542 non-null   int64  
 1   Direction       7542 non-null   object 
 2   expected hits   7542 non-null   float64
 3   rpc_number      5388 non-null   object 
 4   midpoint        5388 non-null   float64
 5   hit_difference  5388 non-null   float64
 6   TDC Number      7542 non-null   int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 412.6+ KB
None


In [139]:
tdc_numbers = combined_df['TDC Number'].unique()  

tdc_dfs = {}

for tdc_number in tdc_numbers:
    tdc_dfs[tdc_number] = combined_df[combined_df['TDC Number'] == tdc_number]

df_tdc_0 = tdc_dfs[0]

df_tdc_1 = tdc_dfs[1]
df_tdc_2 = tdc_dfs[2]
df_tdc_3 = tdc_dfs[3]
df_tdc_4 = tdc_dfs[4]

In [115]:
show(df_tdc_0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [140]:
thresholds = 3
df_tdc = [df_tdc_0, df_tdc_1, df_tdc_2, df_tdc_3, df_tdc_4]

efficiencies = []

label_tdc = ['tdc0', 'tdc1', 'tdc2', 'tdc3', 'tdc4']

for i in range(5):
    valid_hits = df_tdc[i].dropna(subset=['hit_difference'])
    count_valid = valid_hits[valid_hits['hit_difference'].abs() <= thresholds].shape[0]
    total_valid = df_tdc[i].shape[0]
    efficiency = count_valid / total_valid if total_valid > 0 else 0
    efficiencies.append(efficiency)

plt.figure(figsize=(10, 6))
plt.bar(label_tdc, efficiencies, color='blue')
plt.title('Efficiency vs TDC Number for ETA Direction any 5 AtlasON')
plt.xlabel('TDC Number')
plt.ylabel('Efficiency')
plt.ylim(0, 1)  # Set y-axis limits to show from 0 to 100% efficiency
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()